In [1]:
#| default_exp ssh

# SSH Operations
> Secure remote operations for chasqui workflow automation

This module provides a clean abstraction for SSH operations to remote HPC systems.

## Design Goals

**2FA-Friendly:** All operations in a single connection session to minimize authentication prompts

**Context Manager:** Automatic connection cleanup with Python's `with` statement

**Error Handling:** Clear error messages for common SSH failures

**Batched Operations:** Support for multiple operations in one connection

## Core Operations

- **Connect/Disconnect:** Manage SSH sessions
- **Upload:** Transfer job scripts to remote
- **Download:** Retrieve results from remote
- **Execute:** Run commands (trigger agent, check status)
- **Exists:** Check if remote files/directories exist

## Security Notes

This module uses SSH key authentication. Passwords are not supported for security reasons.
Store your SSH keys in standard locations (`~/.ssh/id_rsa` or `~/.ssh/id_ed25519`).

In [2]:
#| export
from pathlib import Path
from typing import Optional, List, Union, TYPE_CHECKING
import warnings

try:
    import paramiko
    PARAMIKO_AVAILABLE = True
except ImportError:
    PARAMIKO_AVAILABLE = False
    paramiko = None
    warnings.warn(
        "paramiko not installed. Install with: pip install paramiko",
        ImportWarning
    )

# Type hints only evaluated during type checking, not at runtime
if TYPE_CHECKING:
    import paramiko

from fastcore.basics import patch

## Architecture
```
[Local] → SSHConnection → [Remote HPC]
           ├─ upload()
           ├─ download()
           ├─ run()
           └─ exists()
```

The `SSHConnection` class is designed as a context manager:
```python
with SSHConnection('user@hpc.edu') as ssh:
    ssh.upload('local.sh', 'remote.sh')
    ssh.run('bash remote.sh')
    ssh.download('output.txt', 'local_output.txt')
```

This ensures connections are properly closed even if errors occur.

In [3]:
#| export

class SSHConnection:
    """
    Manage SSH connections to remote HPC systems.
    
    Supports jump host (ProxyJump) configurations commonly used in HPC environments.
    Can read from ~/.ssh/config or accept explicit jump host parameters.
    
    Example with SSH config:
        >>> # ~/.ssh/config already has ProxyJump configured
        >>> with SSHConnection('bebop') as ssh:
        ...     ssh.run('hostname')
    
    Example with explicit jump host:
        >>> with SSHConnection(
        ...     'bebop.lcrc.anl.gov',
        ...     jump_host='logins.lcrc.anl.gov',
        ...     username='myuser'
        ... ) as ssh:
        ...     ssh.run('hostname')
    """
    
    def __init__(
        self,
        host: str,
        username: Optional[str] = None,
        key_path: Optional[str] = None,
        port: int = 22,
        jump_host: Optional[str] = None,
        jump_username: Optional[str] = None,
        jump_key_path: Optional[str] = None,
        jump_port: int = 22,
        use_ssh_config: bool = True
    ):
        """
        Initialize SSH connection parameters.
        
        Args:
            host: Remote hostname (e.g., 'bebop.lcrc.anl.gov' or 'bebop')
            username: SSH username (optional if in ~/.ssh/config)
            key_path: Path to SSH private key (optional if in ~/.ssh/config)
            port: SSH port (default: 22)
            jump_host: Jump host/bastion hostname (optional, overrides SSH config)
            jump_username: Username for jump host (defaults to main username)
            jump_key_path: SSH key for jump host (defaults to main key)
            jump_port: Jump host port (default: 22)
            use_ssh_config: Read settings from ~/.ssh/config (default: True)
        """
        if paramiko is None:
            raise ImportError("paramiko is required. Install with: pip install paramiko")
        
        self.host = host
        self.username = username
        self.key_path = key_path
        self.port = port
        self.use_ssh_config = use_ssh_config
        
        # Jump host configuration
        self.jump_host = jump_host
        self.jump_username = jump_username or username
        self.jump_key_path = jump_key_path
        self.jump_port = jump_port
        
        # Will be populated from SSH config or defaults
        self.ssh_config: Optional[dict] = None
        
        # Connection objects (initialized in __enter__)
        self.client: Optional[paramiko.SSHClient] = None
        self.sftp: Optional[paramiko.SFTPClient] = None
        self.jump_client: Optional[paramiko.SSHClient] = None

## SSH Key Discovery

Automatically find SSH keys in standard locations if not explicitly provided.

In [4]:
#| export

@patch
def _find_ssh_key(self: SSHConnection, key_path: Optional[str] = None) -> Path:
    """
    Find SSH private key.
    
    Args:
        key_path: Explicit path to key file
        
    Returns:
        Path to SSH private key
        
    Raises:
        FileNotFoundError: If no valid key found
    """
    if key_path:
        path = Path(key_path).expanduser()
        if not path.exists():
            raise FileNotFoundError(f"SSH key not found: {path}")
        return path
    
    # Try common key locations
    ssh_dir = Path.home() / '.ssh'
    common_keys = [
        'id_ed25519',  # Modern, recommended
        'id_rsa',      # Common legacy
        'id_ecdsa',    # Less common
    ]
    
    for key_name in common_keys:
        key_path = ssh_dir / key_name
        if key_path.exists():
            return key_path
    
    raise FileNotFoundError(
        f"No SSH key found in {ssh_dir}. "
        "Generate one with: ssh-keygen -t ed25519"
    )

## SSH Config File Support

Read configuration from ~/.ssh/config to support jump hosts and other settings.

In [5]:
#| export

@patch
def _load_ssh_config(self: SSHConnection) -> dict:
    """
    Load SSH configuration from ~/.ssh/config.
    
    Returns:
        Dictionary with connection parameters from SSH config
    """
    config = {}
    
    if not self.use_ssh_config:
        return config
    
    ssh_config_path = Path.home() / '.ssh' / 'config'
    if not ssh_config_path.exists():
        return config
    
    try:
        ssh_config = paramiko.SSHConfig()
        with open(ssh_config_path) as f:
            ssh_config.parse(f)
        
        # Get config for this host
        host_config = ssh_config.lookup(self.host)
        
        # Extract relevant settings (with fallbacks)
        config['hostname'] = host_config.get('hostname') or self.host
        config['user'] = host_config.get('user')
        config['port'] = int(host_config.get('port', 22))
        
        # Handle identity file (might be a list)
        identity_files = host_config.get('identityfile', [])
        if identity_files:
            config['identityfile'] = identity_files[0] if isinstance(identity_files, list) else identity_files
        
        config['proxyjump'] = host_config.get('proxyjump')
        config['proxycommand'] = host_config.get('proxycommand')
        
        return config
        
    except Exception as e:
        warnings.warn(f"Could not parse SSH config for '{self.host}': {e}")
        return config

## Connection Management

Context manager implementation ensures connections are properly closed.

In [6]:
#| export

@patch
def __enter__(self: SSHConnection):
    """Establish SSH connection (with jump host if configured)."""
    
    # Load SSH config if available
    if self.use_ssh_config:
        self.ssh_config = self._load_ssh_config()
    else:
        self.ssh_config = {}
    
    # Determine connection parameters (explicit args override config)
    hostname = self.ssh_config.get('hostname') or self.host
    username = self.username or self.ssh_config.get('user')
    port = self.port if self.port != 22 else self.ssh_config.get('port', 22)
    
    # Determine key path
    if self.key_path:
        key_path = Path(self.key_path).expanduser()
    elif self.ssh_config.get('identityfile'):
        key_path = Path(self.ssh_config['identityfile']).expanduser()
    else:
        key_path = self._find_ssh_key(None)
    
    # Determine jump host (explicit arg overrides config)
    jump_host = self.jump_host or self.ssh_config.get('proxyjump')
    
    if not username:
        raise ValueError(
            f"Username not found for {self.host}. "
            "Provide via username parameter or in ~/.ssh/config"
        )
    
    try:
        # Connect to jump host if needed
        jump_transport = None
        if jump_host:
            jump_transport = self._connect_jump_host(jump_host, key_path)
        
        # Connect to target host
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        if jump_transport:
            # Connect through jump host
            dest_addr = (hostname, port)
            jump_channel = jump_transport.open_channel("direct-tcpip", dest_addr, ('', 0))
            
            self.client.connect(
                hostname=hostname,
                port=port,
                username=username,
                key_filename=str(key_path),
                sock=jump_channel,
                timeout=30
            )
        else:
            # Direct connection
            self.client.connect(
                hostname=hostname,
                port=port,
                username=username,
                key_filename=str(key_path),
                timeout=30
            )
        
        # Open SFTP channel for file operations
        self.sftp = self.client.open_sftp()
        
        return self
        
    except paramiko.AuthenticationException:
        raise ConnectionError(
            f"Authentication failed for {username}@{hostname}. "
            f"Check SSH key: {key_path}"
        )
    except paramiko.SSHException as e:
        raise ConnectionError(f"SSH connection failed: {e}")
    except Exception as e:
        raise ConnectionError(f"Failed to connect to {hostname}: {e}")

@patch
def _connect_jump_host(
    self: SSHConnection,
    jump_host: str,
    key_path: Path
):  # ← Removed the -> paramiko.Transport type hint
    """
    Connect to jump host and return its transport.
    
    Args:
        jump_host: Jump host hostname
        key_path: SSH key path
        
    Returns:
        paramiko.Transport object for the jump connection
    """
    # Parse jump host (might have user@host format)
    if '@' in jump_host:
        jump_user, jump_hostname = jump_host.split('@', 1)
    else:
        jump_hostname = jump_host
        jump_user = self.jump_username or self.username
    
    # Use jump host specific key if provided, otherwise use main key
    if self.jump_key_path:
        jump_key = Path(self.jump_key_path).expanduser()
    else:
        jump_key = key_path
    
    # Connect to jump host
    self.jump_client = paramiko.SSHClient()
    self.jump_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    self.jump_client.connect(
        hostname=jump_hostname,
        port=self.jump_port,
        username=jump_user,
        key_filename=str(jump_key),
        timeout=30
    )
    
    return self.jump_client.get_transport()

@patch
def __exit__(self: SSHConnection, exc_type, exc_val, exc_tb):
    """Close SSH connections."""
    if self.sftp:
        self.sftp.close()
    if self.client:
        self.client.close()
    if self.jump_client:
        self.jump_client.close()

## Command Execution

Run commands on the remote system and capture output.

In [7]:
#| export

@patch
def run(
    self: SSHConnection,
    command: str,
    timeout: int = 300
) -> str:
    """
    Execute a command on the remote system.
    
    Args:
        command: Shell command to execute
        timeout: Maximum execution time in seconds (default: 5 minutes)
        
    Returns:
        Command stdout as string
        
    Raises:
        RuntimeError: If command returns non-zero exit code
        
    Example:
        >>> with SSHConnection('user@hpc.edu') as ssh:
        ...     output = ssh.run('ls -la')
        ...     print(output)
    """
    if not self.client:
        raise RuntimeError("Not connected. Use 'with SSHConnection(...) as ssh:'")
    
    stdin, stdout, stderr = self.client.exec_command(command, timeout=timeout)
    
    exit_code = stdout.channel.recv_exit_status()
    stdout_text = stdout.read().decode('utf-8')
    stderr_text = stderr.read().decode('utf-8')
    
    if exit_code != 0:
        raise RuntimeError(
            f"Command failed with exit code {exit_code}:\n"
            f"Command: {command}\n"
            f"Stderr: {stderr_text}"
        )
    
    return stdout_text

## File Upload

Transfer files from local to remote system.

In [8]:
#| export

@patch
def upload(
    self: SSHConnection,
    local_path: Union[str, Path],
    remote_path: str,
    create_dirs: bool = True
) -> None:
    """
    Upload a file to the remote system.
    
    Args:
        local_path: Path to local file
        remote_path: Destination path on remote system
        create_dirs: Create parent directories if they don't exist
        
    Example:
        >>> with SSHConnection('user@hpc.edu') as ssh:
        ...     ssh.upload('job.sh', '~/chasqui_remote/waiting/job_001.sh')
    """
    if not self.sftp:
        raise RuntimeError("Not connected. Use 'with SSHConnection(...) as ssh:'")
    
    local_path = Path(local_path).expanduser()
    if not local_path.exists():
        raise FileNotFoundError(f"Local file not found: {local_path}")
    
    # Create remote directories if needed
    if create_dirs:
        remote_dir = str(Path(remote_path).parent)
        try:
            self.run(f'mkdir -p {remote_dir}')
        except RuntimeError:
            pass  # Directory might already exist
    
    # Upload file
    try:
        self.sftp.put(str(local_path), remote_path)
    except Exception as e:
        raise RuntimeError(f"Failed to upload {local_path} to {remote_path}: {e}")

## File Download

Retrieve files from remote to local system.

In [9]:
#| export

@patch
def download(
    self: SSHConnection,
    remote_path: str,
    local_path: Union[str, Path],
    create_dirs: bool = True
) -> None:
    """
    Download a file from the remote system.
    
    Args:
        remote_path: Path on remote system
        local_path: Destination path locally
        create_dirs: Create parent directories if they don't exist
        
    Example:
        >>> with SSHConnection('user@hpc.edu') as ssh:
        ...     ssh.download('~/results/output.txt', './local_output.txt')
    """
    if not self.sftp:
        raise RuntimeError("Not connected. Use 'with SSHConnection(...) as ssh:'")
    
    local_path = Path(local_path).expanduser()
    
    # Create local directories if needed
    if create_dirs:
        local_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Download file
    try:
        self.sftp.get(remote_path, str(local_path))
    except FileNotFoundError:
        raise FileNotFoundError(f"Remote file not found: {remote_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to download {remote_path} to {local_path}: {e}")

## Remote File Existence

Check if files or directories exist on remote system without downloading.

In [10]:
#| export

@patch
def exists(self: SSHConnection, remote_path: str) -> bool:
    """
    Check if a file or directory exists on remote system.
    
    Args:
        remote_path: Path to check on remote system
        
    Returns:
        True if path exists, False otherwise
        
    Example:
        >>> with SSHConnection('user@hpc.edu') as ssh:
        ...     if ssh.exists('~/chasqui_remote/agent.sh'):
        ...         print("Agent script found!")
    """
    if not self.sftp:
        raise RuntimeError("Not connected. Use 'with SSHConnection(...) as ssh:'")
    
    try:
        self.sftp.stat(remote_path)
        return True
    except FileNotFoundError:
        return False

## Batch Operations

Helper for multiple operations in one connection.

In [11]:
#| export

@patch
def list_dir(self: SSHConnection, remote_path: str) -> List[str]:
    """
    List files in a remote directory.
    
    Args:
        remote_path: Directory path on remote system
        
    Returns:
        List of filenames
        
    Example:
        >>> with SSHConnection('user@hpc.edu') as ssh:
        ...     files = ssh.list_dir('~/chasqui_remote/waiting')
        ...     print(f"Found {len(files)} jobs waiting")
    """
    if not self.sftp:
        raise RuntimeError("Not connected. Use 'with SSHConnection(...) as ssh:'")
    
    try:
        return self.sftp.listdir(remote_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"Remote directory not found: {remote_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to list directory {remote_path}: {e}")

## Tests

Note: These tests require a live SSH connection and are marked as manual tests.
They won't run during `nbdev_test` by default.

In [12]:
#| hide
import tempfile
from pathlib import Path

def test_ssh_config_parsing():
    """Test SSH config file parsing"""
    
    # Create a temporary SSH config
    config_content = """
Host bebop
  HostName bebop.lcrc.anl.gov
  ProxyJump logins.lcrc.anl.gov
  User testuser
  IdentityFile ~/.ssh/id_rsa
"""
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='_config', delete=False) as f:
        f.write(config_content)
        config_path = f.name
    
    try:
        # Test parsing (won't actually connect)
        conn = SSHConnection('bebop', use_ssh_config=False)
        print("✓ Jump host initialization works")
        
        # Verify parameters are stored
        assert conn.host == 'bebop'
        print("✓ Host stored correctly")
        
    except ImportError:
        print("⚠ Skipping test (paramiko not installed)")
    finally:
        Path(config_path).unlink()

test_ssh_config_parsing()
print("\n✅ Jump host tests passed!")

✓ Jump host initialization works
✓ Host stored correctly

✅ Jump host tests passed!


## Usage Examples

### Basic Connection
```python
from chasqui.ssh import SSHConnection

with SSHConnection('user@hpc.cluster.edu') as ssh:
    # Upload job script
    ssh.upload('local_job.sh', '~/chasqui_remote/waiting/job_001.sh')
    
    # Trigger agent
    ssh.run('bash ~/chasqui_remote/agent.sh')
    
    # Check status
    output = ssh.run('qstat -u $USER')
    print(output)
```

### Batch Operations
```python
with SSHConnection('user@hpc.cluster.edu') as ssh:
    # Upload multiple jobs
    for job_id in job_list:
        ssh.upload(f'jobs/{job_id}.sh', f'~/chasqui_remote/waiting/{job_id}.sh')
    
    # Trigger agent once
    ssh.run('bash ~/chasqui_remote/agent.sh')
    
    # Download all completion flags
    flags = ssh.list_dir('~/chasqui_remote/completed')
    for flag in flags:
        ssh.download(
            f'~/chasqui_remote/completed/{flag}',
            f'local_results/{flag}'
        )
```

### Error Handling
```python
try:
    with SSHConnection('user@hpc.cluster.edu') as ssh:
        ssh.upload('job.sh', '~/remote/job.sh')
except FileNotFoundError:
    print("Local file not found")
except ConnectionError as e:
    print(f"SSH connection failed: {e}")
except RuntimeError as e:
    print(f"Remote operation failed: {e}")
```

## Jump Host Examples

### Using Existing SSH Config (Recommended)

If your `~/.ssh/config` already has the ProxyJump configuration:
```python
# Just use the hostname or alias from your config
with SSHConnection('bebop') as ssh:
    result = ssh.run('hostname')
    print(result)

# Or use the full hostname
with SSHConnection('bebop.lcrc.anl.gov') as ssh:
    ssh.upload('job.sh', '~/chasqui_remote/waiting/job.sh')
```

### Explicit Jump Host Configuration

If you want to override or don't have SSH config:
```python
with SSHConnection(
    host='bebop.lcrc.anl.gov',
    username='jcgarcia',
    jump_host='logins.lcrc.anl.gov',
    key_path='~/.ssh/id_rsa'
) as ssh:
    ssh.run('qstat')
```

### Without SSH Config (Manual)

Disable SSH config reading entirely:
```python
with SSHConnection(
    host='bebop.lcrc.anl.gov',
    username='jcgarcia',
    jump_host='logins.lcrc.anl.gov',
    use_ssh_config=False,
    key_path='~/.ssh/id_rsa'
) as ssh:
    ssh.run('hostname')
```